In [ ]:
import os
import pandas as pd

In [12]:
uci_ids = {
    "adult": 2,
    "contrac": 23,           # Contraceptive Method Choice
    "covertype": 31,
    "fertility": 244,
    "hill-valey": 606,
    "ht-sensor": 362,        # Gas sensors for home activity monitoring
    "iris": 53,
    "letter": 59,
    "libras": 181,
    "minibooone": 199,
    "skin": 229,
    "susy": 279,
    "wdbc": 17,              # Wisconsin Diagnostic Breast Cancer
    "wine": 109,
    "wine-red": 186,
    "wine-white": 187,
}

datasets = list(uci_ids.keys())